In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import  os
os.chdir('/content/drive/MyDrive/Mask_RCNN-master/')

In [ ]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [ ]:
import tensorflow
print(tensorflow.__version__)

1.15.2


In [ ]:
pip install twilio

     |████████████████████████████████| 481kB 7.5MB/s 
  Created wheel for twilio: filename=twilio-6.59.0-py2.py3-none-any.whl size=1268922 sha256=14d42cc63cac8757117b680390eb48b73ace30202534f6a1ff0de735bcaa3d7e
  Stored in directory: /root/.cache/pip/wheels/8d/eb/43/33d7e4374b4d83eca3d7563fdf06ce55462f4c0b5adc84d2ef
Successfully built twilio


In [ ]:
import sys
import random
import math
import numpy as np
import skimage.io
import matplotlib as pt
import matplotlib.pyplot as plt
import cv2
import mrcnn.config
import mrcnn.utils
from mrcnn.model import MaskRCNN
import imutils
from pathlib import Path
from twilio.rest import Client

Using TensorFlow backend.


In [ ]:

# Twilio config
twilio_account_sid = ' '
twilio_auth_token = ' '
twilio_phone_number = ' '
destination_phone_number = ' '
client = Client(twilio_account_sid, twilio_auth_token)

In [ ]:
# Mask-RCNN config
class MaskRCNNConfig(mrcnn.config.Config):
    NAME = "coco_pretrained_model_config"
    IMAGES_PER_GPU = 1
    GPU_COUNT = 1
    NUM_CLASSES = 1 + 80  
    DETECTION_MIN_CONFIDENCE = 0.6 

# Root dir
ROOT_DIR = Path(".")

#Trained model loc
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")

#if not os.path.exists(COCO_MODEL_PATH):
 #   mrcnn.utils.download_trained_weights(COCO_MODEL_PATH)



model = MaskRCNN(mode="inference", model_dir=MODEL_DIR, config=MaskRCNNConfig())

model.load_weights(COCO_MODEL_PATH, by_name=True)
VIDEO_SOURCE = "./video.mp4"
video_capture = cv2.VideoCapture(VIDEO_SOURCE)

# spotted parking spaces
parked_car_boxes = None



free_space_frames = False
sms_sent = False
count = 0
temp = np.array(4,)


# Filter to only cars
def get_car_boxes(boxes, class_ids):
    car_boxes = []

    for i, box in enumerate(boxes):
        if class_ids[i] in [3, 8, 6]:
            car_boxes.append(box)

    return np.array(car_boxes)

#parked_car_boxes1 = [None] * 11


while video_capture.isOpened():
    success, frame = video_capture.read()
    
    #image = frame
    if not success:
        print("couldn't read video")
        break
        
    elif count<60:
      #create another video reader object to compare the two frames and verify the possibility of motion
      success, frame2 = video_capture.read()
      d = cv2.absdiff(frame, frame2)  
      grey = cv2.cvtColor(d, cv2.COLOR_BGR2GRAY)
      blur = cv2.GaussianBlur(grey, (1, 1), 0)
      ret, th = cv2.threshold( blur, 20, 255, cv2.THRESH_BINARY)
      
      #perform these morphological transformations to erode the car which is moving so that it is not detected by MASKRCNN. Take the eorsion levels to be high. 
      dilated = cv2.dilate(th, np.ones((10, 10), np.uint8), iterations=1 )
      eroded = cv2.erode(dilated, np.ones((30, 30), np.uint8), iterations=1 )
        
      #fill the contours for even a better morphing of the vehicle
      c, h = cv2.findContours(eroded, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
      frame2 = cv2.drawContours(frame2, c, -1, (0,0,0), cv2.FILLED)
      
 

       
      if count%5 == 0:
        print("Current frame counter" + str(count))
        pt.pyplot.imshow(frame2)
        pt.pyplot.show()
        
      count = count + 1
      continue
    
    # Converting the image from BGR color used by OpenCV to RGB color. 
    rgb_image=frame[:, :,::-1]
     
    
    if parked_car_boxes is None:
        print("going in to mark vehicles. Frame number:  ", count)
       
        # This is the first frame of video - assume all the cars detected are in parking spaces.
        results = model.detect([rgb_image], verbose=0)
        r = results[0]
        parked_car_boxes = get_car_boxes(r['rois'], r['class_ids'])
          
          
    elif len(parked_car_boxes)!=0:
        results = model.detect([rgb_image], verbose=0)
        r = results[0]
        # Get where cars are currently located in the frame
        car_boxes = get_car_boxes(r['rois'], r['class_ids'])

        # See how much those cars overlap with the known parking spaces
        overlaps = mrcnn.utils.compute_overlaps(parked_car_boxes, car_boxes)

    
        # Loop through each known parking space box
        for parking_area, overlap_areas in zip(parked_car_boxes, overlaps):

            # For this parking space, find the max amount it was covered by any
            # car that was detected in our image (doesn't really matter which car)
            max_IoU_overlap = np.max(overlap_areas)

            # Get the top-left and bottom-right coordinates of the parking area
            y1, x1, y2, x2 = parking_area

            # Check if the parking space is occupied by seeing if any car overlaps
            # it by more than 0.15 using IoU
            if max_IoU_overlap < 0.15:
                # Parking space not occupied! Draw a green box around it
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 3)
                # Flag that we have seen at least one open space
                free_space = True
            else:
                # Parking space is still occupied - draw a red box around it
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 1)

            # Write the IoU measurement inside the box
            font = cv2.FONT_HERSHEY_DUPLEX
            cv2.putText(frame, f"{max_IoU_overlap:0.2}", (x1 + 6, y2 - 6), font, 0.3, (255, 255, 255))

        # If at least one space was free, start counting frames
        # This is so we don't alert based on one frame of a spot being open.
        # This helps prevent the script triggered on one bad detection.
    
        if free_space:
            free_space_frames += 1
        else:
            # If no spots are free, reset the count
            free_space_frames = 0

        # If a space has been free for several frames, we are pretty sure it is really free!
        if free_space_frames > 2:
                     
            # If we haven't sent an SMS yet, sent it!
            if not sms_sent:
                print("SENDING SMS!!!")
                message = client.messages.create(
                    body="Parking space available!!!",
                    from_=twilio_phone_number,
                    to=destination_phone_number,
                )
                sms_sent = True
                print("Hope you got the message on your phone")
  
        # Show the frame of video on the screen
#         cv2.imshow('Video', frame)
    #saving each frame
    name = str(count) + ".jpg"
    name = os.path.join('/content/drive/MyDrive/Mask_RCNN-master/tabassum', name)
    cv2.imwrite(name, frame)
    count+=1
    
   

video_capture.release()
# cv2.destroyAllWindows()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
box_ind is deprecated, use box_indices instead


Instructions for updating:
Use `tf.cast` instead.


AttributeError: ignored